In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Dependencies
import pytz
import datetime as dt
from datetime import datetime
import dateparser
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from pprint import pprint 
from scipy.stats import linregress
import scipy.stats as stats
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from suntime import Sun, SunTimeException
import os
import json
import urllib.request


In [6]:
accident_data_to_load ="../Resources/US_Accidents_May19.csv"
accidents_data_df = pd.read_csv(
    accident_data_to_load)

# Visualize
accidents_data_df = accidents_data_df.reset_index()
accidents_data_df.set_index('ID', inplace=True)
accidents_data_df.head()


,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,29.68,10.0,Calm,NaN,0.02,Light Rain,False,Night,Night,Night
A-2,1,OH,2,2/8/16 6:07,2/8/16 6:37,39.928059,-82.831184,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,2584.0,...,29.65,10.0,Calm,NaN,0.00,Light Rain,False,Night,Night,Day
A-3,2,OH,2,2/8/16 6:49,2/8/16 7:19,39.063148,-84.032608,0.01,Accident on OH-32 State Route 32 Westbound at ...,NaN,...,29.67,10.0,SW,3.5,NaN,Overcast,True,Night,Night,Day
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,29.64,9.0,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day
A-5,4,OH,2,2/8/16 7:39,2/8/16 8:09,39.627781,-84.188354,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,NaN,...,29.65,6.0,SW,3.5,NaN,Mostly Cloudy,True,Day,Day,Day


In [7]:
#Taking only the interstates
accidents_data_df['hwy'] = accidents_data_df['Street'].str.split("-", n = 1, expand = True)[0].str.contains("I")== True
accidents_data_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,10.0,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,True
A-2,1,OH,2,2/8/16 6:07,2/8/16 6:37,39.928059,-82.831184,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,2584.0,...,10.0,Calm,NaN,0.00,Light Rain,False,Night,Night,Day,False
A-3,2,OH,2,2/8/16 6:49,2/8/16 7:19,39.063148,-84.032608,0.01,Accident on OH-32 State Route 32 Westbound at ...,NaN,...,10.0,SW,3.5,NaN,Overcast,True,Night,Night,Day,False
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,9.0,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,True
A-5,4,OH,2,2/8/16 7:39,2/8/16 8:09,39.627781,-84.188354,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,NaN,...,6.0,SW,3.5,NaN,Mostly Cloudy,True,Day,Day,Day,False


In [8]:
interstate_df = accidents_data_df.loc[accidents_data_df['hwy'] == True]
interstate_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,10.0,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,True
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,9.0,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,True
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,3.0,SSW,4.6,0.02,Light Rain,False,Day,Day,Day,True
A-27,26,OH,2,2/8/16 12:52,2/8/16 13:37,39.733219,-84.159653,0.00,Accident on Irving Ave at Wilmington Ave.,1240.0,...,7.0,West,9.2,0.00,Light Snow,True,Day,Day,Day,True
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,4.0,SW,11.5,NaN,Light Snow,False,Night,Night,Day,True


In [9]:
#Extract last letter of a string from a Pandas column to get the direction
interstate_df['Direction'] = interstate_df['Street'].str.strip().str[-1]
interstate_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,True,E
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,True,S
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,SSW,4.6,0.02,Light Rain,False,Day,Day,Day,True,E
A-27,26,OH,2,2/8/16 12:52,2/8/16 13:37,39.733219,-84.159653,0.00,Accident on Irving Ave at Wilmington Ave.,1240.0,...,West,9.2,0.00,Light Snow,True,Day,Day,Day,True,e
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,SW,11.5,NaN,Light Snow,False,Night,Night,Day,True,S


In [10]:
interstate_df['hwy'] = interstate_df['Direction'].str.contains("E|W|N|S")== True
interstate_df.head()


,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,True,E
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,True,S
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,SSW,4.6,0.02,Light Rain,False,Day,Day,Day,True,E
A-27,26,OH,2,2/8/16 12:52,2/8/16 13:37,39.733219,-84.159653,0.00,Accident on Irving Ave at Wilmington Ave.,1240.0,...,West,9.2,0.00,Light Snow,True,Day,Day,Day,False,e
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,SW,11.5,NaN,Light Snow,False,Night,Night,Day,True,S


In [11]:
hwy_df = interstate_df.loc[interstate_df['hwy'] == True]
hwy_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,True,E
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,True,S
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,SSW,4.6,0.02,Light Rain,False,Day,Day,Day,True,E
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,SW,11.5,NaN,Light Snow,False,Night,Night,Day,True,S
A-66,65,OH,3,2/9/16 8:23,2/9/16 8:53,39.622787,-84.229858,0.01,Accident on I-75 Southbound at I-675. Expect d...,NaN,...,WSW,9.2,0.00,Light Snow,False,Day,Day,Day,True,N


In [12]:
#converting the direction column to a list to check we get only teh valid direstions, N S E W
directionToList = hwy_df['Direction'].tolist()
#print(directionToList)

In [13]:
len(hwy_df)

209060

In [14]:
#extracting the accident date
hwy_df['Accident_Date'] = hwy_df['Start_Time'].str.split(" ", n = 1, expand = True)[0]
hwy_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction,Accident_Date
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,NaN,0.02,Light Rain,False,Night,Night,Night,True,E,2/8/16
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,True,S,2/8/16
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,4.6,0.02,Light Rain,False,Day,Day,Day,True,E,2/8/16
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,11.5,NaN,Light Snow,False,Night,Night,Day,True,S,2/9/16
A-66,65,OH,3,2/9/16 8:23,2/9/16 8:53,39.622787,-84.229858,0.01,Accident on I-75 Southbound at I-675. Expect d...,NaN,...,9.2,0.00,Light Snow,False,Day,Day,Day,True,N,2/9/16


In [15]:
#Converting date from string format to date format
hwy_df['Accident_Date'] = pd.to_datetime(hwy_df['Accident_Date'])
hwy_df['Accident_Date_No_Time'] = hwy_df['Accident_Date'].dt.date
hwy_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction,Accident_Date,Accident_Date_No_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,0.02,Light Rain,False,Night,Night,Night,True,E,2016-02-08,2016-02-08
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,NaN,Mostly Cloudy,False,Night,Day,Day,True,S,2016-02-08,2016-02-08
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,0.02,Light Rain,False,Day,Day,Day,True,E,2016-02-08,2016-02-08
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,NaN,Light Snow,False,Night,Night,Day,True,S,2016-02-09,2016-02-09
A-66,65,OH,3,2/9/16 8:23,2/9/16 8:53,39.622787,-84.229858,0.01,Accident on I-75 Southbound at I-675. Expect d...,NaN,...,0.00,Light Snow,False,Day,Day,Day,True,N,2016-02-09,2016-02-09


In [16]:
hwy_df = hwy_df.drop(['hwy', 'Accident_Date'], axis=1)

In [17]:
hwy_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date_No_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,E,2016-02-08
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,S,2016-02-08
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,SSW,4.6,0.02,Light Rain,False,Day,Day,Day,E,2016-02-08
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,SW,11.5,NaN,Light Snow,False,Night,Night,Day,S,2016-02-09
A-66,65,OH,3,2/9/16 8:23,2/9/16 8:53,39.622787,-84.229858,0.01,Accident on I-75 Southbound at I-675. Expect d...,NaN,...,WSW,9.2,0.00,Light Snow,False,Day,Day,Day,N,2016-02-09


In [18]:
hwy_df = hwy_df.rename({'Accident_Date_No_Time':'Accident_Date'}, axis=1)
hwy_df.head()
#hwy_df['Accident_Time'] = hwy_df['Start_Time'].str.split(" ", n = 1, expand = True)[1]


,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,E,2016-02-08
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,S,2016-02-08
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,SSW,4.6,0.02,Light Rain,False,Day,Day,Day,E,2016-02-08
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,SW,11.5,NaN,Light Snow,False,Night,Night,Day,S,2016-02-09
A-66,65,OH,3,2/9/16 8:23,2/9/16 8:53,39.622787,-84.229858,0.01,Accident on I-75 Southbound at I-675. Expect d...,NaN,...,WSW,9.2,0.00,Light Snow,False,Day,Day,Day,N,2016-02-09


In [19]:
hwy_df['Accident_Time'] = hwy_df['Start_Time'].str.split(" ", n = 1, expand = True)[1]
hwy_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date,Accident_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,NaN,0.02,Light Rain,False,Night,Night,Night,E,2016-02-08,5:46
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,S,2016-02-08,7:23
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,4.6,0.02,Light Rain,False,Day,Day,Day,E,2016-02-08,8:21
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,11.5,NaN,Light Snow,False,Night,Night,Day,S,2016-02-09,6:41
A-66,65,OH,3,2/9/16 8:23,2/9/16 8:53,39.622787,-84.229858,0.01,Accident on I-75 Southbound at I-675. Expect d...,NaN,...,9.2,0.00,Light Snow,False,Day,Day,Day,N,2016-02-09,8:23


In [40]:
hwy_rush_hour_df = hwy_df.copy()
hwy_non_rush_hour_df = hwy_df.copy()
hwy_rush_hour_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date,Accident_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,NaN,0.02,Light Rain,False,Night,Night,Night,E,2016-02-08,5:46
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,S,2016-02-08,7:23
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,4.6,0.02,Light Rain,False,Day,Day,Day,E,2016-02-08,8:21
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,11.5,NaN,Light Snow,False,Night,Night,Day,S,2016-02-09,6:41
A-66,65,OH,3,2/9/16 8:23,2/9/16 8:53,39.622787,-84.229858,0.01,Accident on I-75 Southbound at I-675. Expect d...,NaN,...,9.2,0.00,Light Snow,False,Day,Day,Day,N,2016-02-09,8:23


In [41]:
rush_hour_accidents = []

#rush hour is morning 7 to 10 am and evening 4 to 7 pm
start_morning = dt.time(7, 0)
end_morning = dt.time(10, 0) 
start_evening = dt.time(16, 0)
end_evening = dt.time(19, 0)


for accident_time in hwy_rush_hour_df.loc[:, 'Accident_Time']:
    #print(accident_time)
    accident_time_obj = datetime.strptime(accident_time, '%H:%M').time()
    #print(accident_time_obj)
    if(start_morning < accident_time_obj < end_morning) or (start_evening < accident_time_obj < end_evening):
        rush_hour_accidents.append(accident_time_obj)
    
print(rush_hour_accidents)
        


[datetime.time(7, 23), datetime.time(8, 21), datetime.time(8, 23), datetime.time(8, 35), datetime.time(18, 41), datetime.time(7, 21), datetime.time(7, 29), datetime.time(8, 29), datetime.time(17, 10), datetime.time(18, 27), datetime.time(7, 2), datetime.time(7, 46), datetime.time(7, 2), datetime.time(7, 14), datetime.time(8, 59), datetime.time(8, 20), datetime.time(8, 39), datetime.time(9, 22), datetime.time(9, 26), datetime.time(17, 16), datetime.time(7, 52), datetime.time(8, 21), datetime.time(16, 30), datetime.time(7, 43), datetime.time(7, 39), datetime.time(16, 15), datetime.time(7, 14), datetime.time(7, 50), datetime.time(16, 36), datetime.time(18, 52), datetime.time(7, 9), datetime.time(7, 27), datetime.time(9, 2), datetime.time(7, 46), datetime.time(18, 41), datetime.time(7, 10), datetime.time(16, 41), datetime.time(18, 34), datetime.time(9, 37), datetime.time(17, 19), datetime.time(17, 28), datetime.time(8, 7), datetime.time(8, 51), datetime.time(17, 28), datetime.time(17, 44),

In [42]:
hwy_rush_hour_df = hwy_rush_hour_df.iloc[:len(rush_hour_accidents), :]
hwy_rush_hour_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date,Accident_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,NaN,0.02,Light Rain,False,Night,Night,Night,E,2016-02-08,5:46
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,S,2016-02-08,7:23
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,4.6,0.02,Light Rain,False,Day,Day,Day,E,2016-02-08,8:21
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,11.5,NaN,Light Snow,False,Night,Night,Day,S,2016-02-09,6:41
A-66,65,OH,3,2/9/16 8:23,2/9/16 8:53,39.622787,-84.229858,0.01,Accident on I-75 Southbound at I-675. Expect d...,NaN,...,9.2,0.00,Light Snow,False,Day,Day,Day,N,2016-02-09,8:23


In [43]:
hwy_rush_hour_df['Rush_Hour_Accident_Time'] = rush_hour_accidents
hwy_rush_hour_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date,Accident_Time,Rush_Hour_Accident_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,0.02,Light Rain,False,Night,Night,Night,E,2016-02-08,5:46,07:23:00
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,NaN,Mostly Cloudy,False,Night,Day,Day,S,2016-02-08,7:23,08:21:00
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,0.02,Light Rain,False,Day,Day,Day,E,2016-02-08,8:21,08:23:00
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,NaN,Light Snow,False,Night,Night,Day,S,2016-02-09,6:41,08:35:00
A-66,65,OH,3,2/9/16 8:23,2/9/16 8:53,39.622787,-84.229858,0.01,Accident on I-75 Southbound at I-675. Expect d...,NaN,...,0.00,Light Snow,False,Day,Day,Day,N,2016-02-09,8:23,18:41:00


In [39]:
len(hwy_rush_hour_df)


34534

In [44]:
non_rush_hour_accidents = []

#non rush hour is between 10am to 4 pm

start = dt.time(10, 0)
end = dt.time(16, 0) 

for accident_time in hwy_non_rush_hour_df.loc[:, 'Accident_Time']:
    #print(accident_time)
    accident_time_obj = datetime.strptime(accident_time, '%H:%M').time()
    #print(accident_time_obj)
    if(start < accident_time_obj < end):
        non_rush_hour_accidents.append(accident_time_obj)
    
print(non_rush_hour_accidents)
        

[datetime.time(10, 6), datetime.time(11, 5), datetime.time(11, 25), datetime.time(10, 41), datetime.time(10, 19), datetime.time(10, 22), datetime.time(11, 27), datetime.time(15, 45), datetime.time(10, 5), datetime.time(14, 55), datetime.time(14, 44), datetime.time(15, 57), datetime.time(10, 48), datetime.time(13, 33), datetime.time(13, 13), datetime.time(12, 24), datetime.time(13, 52), datetime.time(10, 30), datetime.time(10, 49), datetime.time(10, 31), datetime.time(10, 17), datetime.time(11, 16), datetime.time(11, 38), datetime.time(11, 50), datetime.time(12, 6), datetime.time(12, 20), datetime.time(12, 54), datetime.time(13, 2), datetime.time(13, 15), datetime.time(13, 27), datetime.time(13, 59), datetime.time(14, 17), datetime.time(14, 34), datetime.time(15, 55), datetime.time(10, 14), datetime.time(10, 17), datetime.time(10, 33), datetime.time(10, 49), datetime.time(10, 53), datetime.time(11, 5), datetime.time(11, 11), datetime.time(11, 21), datetime.time(11, 19), datetime.time(11

In [45]:
len(non_rush_hour_accidents)

62536

In [47]:
hwy_non_rush_hour_df = hwy_non_rush_hour_df.iloc[:len(non_rush_hour_accidents), :]
hwy_non_rush_hour_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date,Accident_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,NaN,0.02,Light Rain,False,Night,Night,Night,E,2016-02-08,5:46
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,S,2016-02-08,7:23
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,4.6,0.02,Light Rain,False,Day,Day,Day,E,2016-02-08,8:21
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,11.5,NaN,Light Snow,False,Night,Night,Day,S,2016-02-09,6:41
A-66,65,OH,3,2/9/16 8:23,2/9/16 8:53,39.622787,-84.229858,0.01,Accident on I-75 Southbound at I-675. Expect d...,NaN,...,9.2,0.00,Light Snow,False,Day,Day,Day,N,2016-02-09,8:23


In [48]:
hwy_non_rush_hour_df['Non_Rush_Hour_Accident_Time'] = non_rush_hour_accidents
hwy_non_rush_hour_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date,Accident_Time,Non_Rush_Hour_Accident_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,0.02,Light Rain,False,Night,Night,Night,E,2016-02-08,5:46,10:06:00
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,NaN,Mostly Cloudy,False,Night,Day,Day,S,2016-02-08,7:23,11:05:00
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,0.02,Light Rain,False,Day,Day,Day,E,2016-02-08,8:21,11:25:00
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,NaN,Light Snow,False,Night,Night,Day,S,2016-02-09,6:41,10:41:00
A-66,65,OH,3,2/9/16 8:23,2/9/16 8:53,39.622787,-84.229858,0.01,Accident on I-75 Southbound at I-675. Expect d...,NaN,...,0.00,Light Snow,False,Day,Day,Day,N,2016-02-09,8:23,10:19:00


In [50]:
group_directions_rush_hour = hwy_rush_hour_df.groupby(["Direction"], as_index=False)["Rush_Hour_Accident_Time"].count()
group_directions_rush_hour.head()

,Direction,Rush_Hour_Accident_Time
0,E,19122
1,N,32242
2,S,20695
3,W,17224


In [51]:
group_directions_non_rush_hour = hwy_non_rush_hour_df.groupby(["Direction"], as_index=False)["Non_Rush_Hour_Accident_Time"].count()
group_directions_non_rush_hour.head()

,Direction,Non_Rush_Hour_Accident_Time
0,E,13422
1,N,22569
2,S,14535
3,W,12010


In [52]:
stats.ttest_ind(group_directions_rush_hour.Rush_Hour_Accident_Time, group_directions_non_rush_hour.Non_Rush_Hour_Accident_Time, equal_var=False)

Ttest_indResult(statistic=1.6193499743134547, pvalue=0.16224510955685342)

In [ ]:
# bar chart of rush hour evening vs morning vs non rush hour during workur